# EDS 220 Fall 2021

## Visualizing global precipitation using Google Earth Engine

Let's get started with some data visualization and processing! Our tool for this exercise will be the Google Earth Engine (GEE):
https://earthengine.google.com/

GEE is a really powerful platform that allows easy access to many datasets without the need for a lot of pre-processing, so it's a good way to get our hands dirty with some basic mapping skills. Here we'll use the Python API, which will let us code in 'familiar' Python commands while still interfacing with the GEE platform. 

#### Recap from last time (GEE_Setup.ipynb)
Prior to starting this notebook, you should have gone through the following steps:
1. **Activating the Python environment containing your local GEE installation**
   (on the command line in your terminal window) conda activate gee_env  
   where "gee_env" is the name you gave to the environment creates for GEE to live in
     
2. **Authenticating Earth Engine**
   (also on the terminal command line) earthengine authenticate  
   This should bring up a login window in your Internet browser, where after entering your Google account information it will give you an authorization code to enter in the terminal
     
3. **Launching JupyterLab from within the appropriate Python environment**  
   - If starting JupyterLab from the command line: make sure it says "(gee_env)" before your login name. This happens after the "conda activate gee_env" step, and your terminal prompt will look something like:  
   (gee_env) Samanthas-MacBook-Pro:~ samstevenson$    
     
   - If starting JupyterLab from the Anaconda Navigator: make sure to go to the drop-down menu that says "Applications on..." and select "gee_env" before launching JupyterLab


#### Importing Packages
The code chunk below imports the needed packages: 
- **ee**, which covers the Google Earth Engine interface
- **geemap**, which covers the GEE mapping functionality
- **pandas**, to allow for manipulation of data subsets later on

They're all included at the start of the notebook here, since it's good practice to do all of the importing of packages in one place (best at the beginning of the code) so you can easily see what tools are available to Python.

As we discussed in the GEE_Setup notebook, here 'pandas' is being imported with a shorter name 'pd' to make it easier to reference that package later on. This is done with the "import...as" syntax. note that you *could* do this with any of the other packages as well, and you can call them whatever you want!

In [8]:
import ee
import geemap
import pandas as pd

## Create an interactive map

Now let's load the package that will allow us to map a given dataset. The default basemap is, unsurprisingly, Google Maps. The code chunk below will display an empty Google Map that you can manipulate just like a regular Google Maps interface.  

We accomplish this by calling the "Map" function within the geemap package: this uses the standard Python syntax of  
[package name].[function name]  

... or in this case,  
`geemap.Map`  

The output will be stored in an "object", which we will be able to do various things to in subsequent code chunks. (This is standard practice for "object oriented languages" like Python!)  

Also important: below, we are providing an *argument* to the Map function. This is telling it to do two different things: 
- center the map at a particular lat/lon point: in this case, 40N and 100E
- set the *zoom level* to a particular value. Large numbers mean more magnification (i.e. smaller fields of view). Here I set it to 2 since that seemed to be visually pretty appealing, but you can pick whatever you want.

In [6]:
Map = geemap.Map(center=[40,-100], zoom=2)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Load data and plot over the basemap

Now we need to tell GEE which dataset to layer on top of the base map. 

The next chunk of code will load some data; when executed, it will display within the map window above.

#### ERA5 Reanalysis Data
The dataset we'll be using here is the ERA5 "reanalysis": essentially, a reanalysis is a numerical model (in this case, a weather model) that's been forced to match observations of the real world (in this case, atmospheric weather observations) as closely as possible. ERA5 is produced by the European Centre for Medium-Range Weather Forecasts, or ECMWF; so you'll see below that the parent directory for this dataset is called ECMWF. A description of the dataset is here:
https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY

##### Load in ERA5: GEE Image Collections
The below chunk of code loads in the "mean_2m_air_temperature" and "total precipitation" fields, which are basically what they sound like: the temperature of air at an altitude of 2 meters, and the daily total precipitation, everywhere on Earth. Here we'll be working just with precipitation, but I've included temperature as well so you can see that other variables are just as easy to grab!

There are a couple of things to know about how GEE stores image data. The ImageCollection function grabs a *set* of individual images that satisfies some criterion that you pass to GEE through the "ee" package; this is stored as an "ImageCollection" object which can be filtered and processed in various ways later.  

The syntax for this is:  
`ee.ImageCollection([args])`

where you can pass different arguments to tell GEE which ImageCollection you would like. Here I'm going to tell it that I would like all of the ERA5 data: that means passing a location called "ECMWF/ERA5/Daily" that I got by poking through the [Earth Engine data catalog](https://developers.google.com/earth-engine/datasets/). There are also other ERA5 datasets: since we're going to explore precipitation, I'm using daily information so we can see individual rain events.

*Note*: I'm also including comments within the code chunks themselves, noting what they do: this doesn't matter as much within a Jupyter notebook, but if you're writing a Python script, it's important to note what the various lines of code mean for your later self and anyone else who reads your code!

In [16]:
# Add Earth Engine dataset: daily data from ERA5
gdat = ee.ImageCollection('ECMWF/ERA5/DAILY')

#### Select an image to plot

To plot a map over the Google Maps basemap, we need not an "ImageCollection", but an "Image". The following code chunks go through the process of:
- selecting a variable to plot (precipitation)
- filtering that variable to a particular time range
- calculating the average over that time range
- layering the data on top of the basemap

The first thing we need to do is to pick what quantity we would like to plot: ERA5 contains many different climate variables. Below are two examples of selecting a field from an ImageCollection: I'm including two so you can get a sense of how it might work, but we'll be focusing on precipitation since it's more visually interesting.

The selection process is accomplished using the "select" function, applied to the "gdat" object we created in the code chunk above.

**Recall:** the object "gdat" is a *collection* of images, that we will use the `.select` function to pick from:

In [ ]:
# Select temperature and precipitation data from the ERA5 daily images
ts = gdat.select('mean_2m_air_temperature')

pr=gdat.select('total_precipitation')

Let's take a look at the "pr" object for a second. This can be done using the `print` command, or just by simply typing the name of the object:

In [19]:
pr
print(pr)

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_0_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "Image.select",
              "arguments": {
                "bandSelectors": {
                  "constantValue": [
                    "total_precipitation"
                  ]
                },
                "input": {
                  "argumentReference": "_MAPPING_VAR_0_0"
                }
              }
            }
          }
        }
      },
      "collection": {
        "functionInvocationValue": {
          "functionName": "ImageCollection.load",
          "arguments": {
            "id": {
              "constantValue": "ECMWF/ERA5/DAILY"
            }
          }
        }
      }
    }
  }
})


As you can see, `pr` is still an ImageCollection. This makes it not very mappable: in fact, this object contains information every day from 1979 to 2020. In order to create a map, we need to filter it down to a single field for a time of our choice. 

Let's select an arbitrary time range: here, December 1-2, 2019.

The below code might look confusing to start with, so I'll break it down a bit: the first thing it does is applies the Python built-in `filter` command to our precipitation ImageCollection object. `filter` is a standard Python function: essentially, what it does is
- take any object you give it (in this case, "pr")
- apply any arbitrary function to that data

Here, the function we're passing it is confusingly ALSO called filter, but it's the filtering function that's part of the "ee" package. This is `ee.Filter.date`, which takes a range of dates as arguments.

**The point is** that the piece of the code that looks like this:
`pr.filter(ee.Filter.date('2019-12-01', '2019-12-02'))`

is saying "take the pr object, and use Google Earth Engine's date filtering functionality to pick out the bits between December 1 and 2, 2019".

#### Other nifty Python thing: multiple functions applied in one line of code

You'll notice that the line of code below ALSO includes another function: `.mean()`. This is exactly what it sounds like: an averaging operator. `.mean()` is another standard Python built-in, that takes whatever is before the period (in this case, precipitation filtered between December 1-2, 2019) and calculates the average.

**The end result**: In one line of code, we can grab all of the precipitation data in a particular time range, and calculate the average, to get a single image that we can then map!

In [20]:
# Filter temperature and precipitation according to a specified time range, calculate the mean
prdflt=pr.filter(ee.Filter.date('2019-12-01', '2019-12-02')).mean();


### Specify map parameters

Now let's tell the map where to center itself! This is done with the "setCenter" command applied to the Map object. The first two arguments below are the longitude and latitude, respectively; the third is the zoom level for the map (smaller numbers = bigger maps).

In [11]:
# Map.setCenter(-90.162, 29.8597, 4)   # New Orleans, USA
# Map.setCenter(-114.9774, 31.9254, 4) # Mouth of the Colorado River, Mexico
# Map.setCenter(-111.1871, 37.0963, 3) # Lake Powell, USA
# Map.setCenter(149.412, -35.0789, 4)  # Lake George, Australia
Map.setCenter(105.26, 11.2134, 4)     # Mekong River Basin, SouthEast Asia
# Map.setCenter(90.6743, 22.7382, 4)   # Meghna River, Bangladesh
# Map.setCenter(81.2714, 16.5079, 4)   # Godavari River Basin Irrigation Project, India
# Map.setCenter(14.7035, 52.0985, 4)   # River Oder, Germany & Poland
# Map.setCenter(-59.1696, -33.8111, 4)  # Buenos Aires, Argentina\
#Map.setCenter(-74.4557, -8.4289, 4)  # Ucayali River, Peru


Now we can set a color palette to use when plotting the data layer. The following are the palettes specified for temperature and precipitation in the GEE description page for ERA5. These are what are called "Hex" color codes: don't worry about them a huge amount for now, just know that GEE has a lot of color tables like this that you can look up. 

Note that you also have to specify a max/min value for each of the variables!

In [12]:
VIS_TS = {
    'min':273,
    'max':310,
    'palette': ['#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF']
}

VIS_PREC = {
    'min':0,
    'max':0.1,
    'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
}

### Add data to the map

Finally, now that we have our precipitation image `prdflt`, we can put it on top of the basemap! This is done using the `.addLayer` function, which we apply to the `Map` object that contains the Google Maps basemap. We also pass it other *arguments*:
- the visualization parameters (color ranges, maps) stored in VIS_PREC
- the name of the field ('total precipitation')
- the *opacity* so that we can also see the basemap underneath (feel free to play with this one to see how things change!)

In [13]:
Map.addLayer(prdflt, VIS_PREC,'total precipitation',opacity=0.3)

## Play with the map!

Now that everything has been set up, try playing around with things a bit. 

#### 1) Look at time variations

Change the date range in the ee.Filter.date command above, then replot the map layer. How different do things look? 

#### 2) Change map locations

Try zooming in and out on different parts of the map. What types of features do you notice? Are there particular patterns that seem to persist in particular places? (for example: check out the equatorial Pacific!)

## Look for historical storm events

Test out your map manipulation skills! Try filtering the data in time and space to locate a couple of particularly strong precipitation events:

- Montecito, California: January 9, 2018

- Rio Grande do Sul, Brazil: March 11-13, 2017

- Mumbai, India: July 26, 2005

(hint: Google is your friend for finding lats/lons for these places!)

Can you see any coherent patterns of precipitation in those locations on those days?


## Save images of your maps

Finally, save your work! There are a few different options for saving images from GEE; here we'll use the getThumbURL() method. This generates a JPG or PNG image, for which the URL is provided, then you can download the result. 

In [14]:
url = prdflt.getThumbUrl({
    'min': 0, 'max': 0.1, 'dimensions': 512, 
    'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']})
print(url)


https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/1a0117e07448fda243811263823fed20-c54d923cdae299e1253b2d0f747fdb96:getPixels


Once you've executed the above code, you can save it to whatever filename you like!